In [1]:
from word_graph_lib import WordGraph, get_words, open_ipa_dict_representations_map, open_cmudict_representations_map

In [2]:
MAX_WORD_LENGTH = None

In [3]:
words = get_words('12dicts_words.txt')
representations = None
label_rep_function = None

In [4]:
# # We can also use different representations of words.
# # Here, we will use IPA representations from this dictionary https://github.com/open-dict-data/ipa-dict
# # There are also other choices like https://github.com/DanielSWolf/wiki-pronunciation-dict?tab=readme-ov-file, but they don't seem as complete
# # It may be good to combine multiple sources
# whitelisted_words = set(words)
# ipa_dict_representations_map = open_ipa_dict_representations_map(whitelisted_words)
# ipa_words = set(ipa_dict_representations_map.keys())
# missed_words = whitelisted_words - ipa_words
# print(f"Found {len(ipa_words)} words in the IPA dictionary, missed {len(missed_words)} words")

# # Filter down the words and representations to only the ones we have in the IPA dictionary
# ipa_words = []
# ipa_representations = []
# for word, representations in ipa_dict_representations_map.items():
#     for representation in representations:
#         ipa_words.append(word)  # Duplicate words are allowed as long as they have different representations
#         ipa_representations.append(representation)

# words = ipa_words
# representations = ipa_representations

# def label_rep_function(word, representation):
#     return f"{word} ({''.join(representation)})"

In [5]:
# # Here we instead use the CMU pronunciation dictionary which uses ARPAbet instead of IPA
# # ARPAbet is a simpler phonetic alphabet, but it is less accurate. The simplicity may be good for having more connected components
# # The link is here https://github.com/cmusphinx/cmudict
# whitelisted_words = set(words)
# cmudict_representations_map = open_cmudict_representations_map(whitelisted_words)
# cmudict_words = set(cmudict_representations_map.keys())
# missed_words = whitelisted_words - cmudict_words
# print(f"Found {len(cmudict_words)} words in the CMU dictionary, missed {len(missed_words)} words")

# # Filter down the words and representations to only the ones we have in the CMU dictionary
# cmudict_words = []
# cmudict_representations = []
# for word, representations in cmudict_representations_map.items():
#     for representation in representations:
#         cmudict_words.append(word)  # Duplicate words are allowed as long as they have different representations
#         cmudict_representations.append(representation)

# words = cmudict_words
# representations = cmudict_representations

# def label_rep_function(word, representation):
#     return f"{word} ({' '.join(representation)})"

In [6]:
if MAX_WORD_LENGTH:
    allowed_indices = [i for i, word in enumerate(words) if len(word) <= MAX_WORD_LENGTH]
    processed_words = [words[i] for i in allowed_indices]
    processed_representations = [representations[i] for i in allowed_indices]
else:
    processed_words = words
    processed_representations = representations
word_graph = WordGraph(processed_words, representations=processed_representations, label_rep_func=label_rep_function)

print(f"Symbols: {word_graph.symbol_to_int}")

Processing words of length 1
	Number of words of length 1: 4
	Root batch size: 125000 (500000 word pairs per batch)
Processing words of length 2
	Number of words of length 2: 61
	Root batch size: 8196 (499956 word pairs per batch)
Processing words of length 3
	Number of words of length 3: 649
	Root batch size: 770 (499730 word pairs per batch)
Processing words of length 4
	Number of words of length 4: 2446
	Root batch size: 204 (498984 word pairs per batch)
Processing words of length 5
	Number of words of length 5: 4680
	Root batch size: 106 (496080 word pairs per batch)
Processing words of length 6
	Number of words of length 6: 7352
	Root batch size: 68 (499936 word pairs per batch)
Processing words of length 7
	Number of words of length 7: 9878
	Root batch size: 50 (493900 word pairs per batch)
Processing words of length 8
	Number of words of length 8: 10466
	Root batch size: 47 (491902 word pairs per batch)
Processing words of length 9
	Number of words of length 9: 9412
	Root batch 

In [7]:
disconnected_components = word_graph.find_disconnected_components()
print(f"Number of disconnected components: {len(disconnected_components)}")
print(f"Number of disconnected components with more than 2 words: {len([comp for comp in disconnected_components if len(comp) > 2])}")
print(f"Number of disconnected components with more than 5 words: {len([comp for comp in disconnected_components if len(comp) > 5])}")

Number of disconnected components: 26504
Number of disconnected components with more than 2 words: 3024
Number of disconnected components with more than 5 words: 563


In [8]:
path, diameter = word_graph.find_diameter()
str_path = word_graph.path_to_label(path)
print(f"Diameter of the graph: {diameter}")
print(f"Path: {', '.join(str_path)}")

Diameter of the graph: 42
Path: hammerings, hammering, hampering, pampering, papering, capering, catering, cantering, bantering, battering, bettering, fettering, festering, pestering, petering, peering, peeing, seeing, sewing, swing, sing, sine, mine, mire, mere, metre, metred, metered, petered, pestered, festered, fettered, bettered, battered, bantered, cantered, catered, capered, papered, pampered, hampered, hammered, yammered


In [9]:
# Now from cat to orange
# Find the nodes
start_node = 'cat'
end_node = 'orange'

print(f"Shortest path from {start_node} to {end_node}")
path, path_length, _ = word_graph.bfs(start_node, end_node)
if path is None:
    print("No path found")
else:
    str_path = word_graph.path_to_label(path)
    print(f"Path length: {path_length}")
    print(f"Path: {', '.join(str_path)}")

Shortest path from cat to orange
Path length: 5
Path: cat, can, ran, rang, range, orange


In [10]:
# Export the DOT file
dot_string = word_graph.to_dot()
with open('word_graph.dot', 'w') as f:
    f.write(dot_string)